In [1]:
# import libraries
%reload_ext autoreload

from config.Neo_connect import Neo4jConnection
import pandas as pd
import config.db_settings as db_config
from IPython.display import display, HTML

In [2]:
# stablish connection with neo4j
conn = Neo4jConnection(uri="bolt://localhost:7687", user=db_config.USER, pwd=db_config.PASSWORD)


# Graph Analytics

<img src="docs/property_graph.png">

# Sense Stock Pipeline
<img src="docs/Final BDM_PRO_2.png">

## 1. Top Tweet users 

- Finding the most important users for the companies to track, which can influence the stock prices of that company

In [3]:
q = '''match (c:company)<-[:is_related_to]-(t:tweets)-[m:by_user]->(u:user_data)
with c,u, avg(toInteger(t.support_Count)) as avg_support_count, avg(toFloat(t.sentiment)) as avg_sentiment order by avg_support_count DESC
with c, collect([u,avg_support_count,avg_sentiment]) as imp_collection
return c.companyName as company, collect([imp_collection[0][0].name,imp_collection[1][0].name]) as top_users, 
(imp_collection[0][1]+imp_collection[1][1])/2 as support, (imp_collection[0][2]+imp_collection[1][2])/2 as sentiment'''

In [4]:
result = conn.query(q, db=db_config.DATABASE)

In [5]:
df = pd.DataFrame(result)
df.columns=['company','top_users','support_count','sentiment']
display(HTML(df.head(6).to_html()))

,company,top_users,support_count,sentiment
0,ITC Limited,"[[B. Ray, J. Rahul]]",24547.000000,0.000000
1,Reliance Industries Limited,"[[B. Ray, K. Puja]]",13147.000000,0.024330
2,Gland Pharma Limited,"[[R.sharma, E. SharmaAryan]]",5654.166667,0.066071
3,Housing Development Finance Corporation Limited,"[[B. Ray, E. SharmaAryan]]",4268.000000,0.100000
4,Indian Oil Corporation Limited,"[[R.sharma, K. Ashwini]]",2711.500000,0.145536
5,UPL Limited,"[[S. Sanjay, T. Sia]]",865.000000,-0.036607


## 2. Top Mentioned Users 

- Finding the most important users for the companies to track, which can influence the stock prices of that company

In [6]:
q1 = '''CALL gds.graph.project(
'pgmgraph',
['user_data', 'tweets'],
'mentions'
)'''
q2 = '''
CALL gds.pageRank.stream('pgmgraph')
YIELD nodeId,score
with gds.util.asNode(nodeId) AS N, score as page_rank_score
match (N:user_data)
return N.name, page_rank_score order by page_rank_score desc
'''

In [7]:
graph_creation = conn.query(q1, db=db_config.DATABASE)
result = conn.query(q2, db=db_config.DATABASE)

In [8]:
df = pd.DataFrame(result)
df.columns=['Mentioned user name','page_rank_score']
display(HTML(df.head(6).to_html()))

,Mentioned user name,page_rank_score
0,zomato,2.415857
1,narendramodi,0.660000
2,tcs,0.532500
3,vimsicle,0.532500
4,zomatocare,0.532500
5,Adityaaa,0.405000


## 3. Most Influential Companies

- Finding the current top companies to buy and sell stocks of, companies with higher poplarity

In [9]:
q1 = '''CALL gds.graph.project(
'centerGraph',
['company','tweets'],
['belongs_to','is_related_to']
)'''

q2 = '''
CALL gds.pageRank.stream('centerGraph')
YIELD nodeId, score
with gds.util.asNode(nodeId) AS N, score
match (N:company)
return N.companyName as company , score as page_rank_score 
order by page_rank_score desc
'''

In [10]:
graph_creation = conn.query(q1, db=db_config.DATABASE)
result = conn.query(q2, db=db_config.DATABASE)

In [11]:
df = pd.DataFrame(result)
df.columns=['company','page_rank_score']
display(HTML(df.head(6).to_html()))

,company,page_rank_score
0,Reliance Industries Limited,7.6725
1,ITC Limited,4.2300
2,Gland Pharma Limited,3.9750
3,Zomato Limited,3.8475
4,Housing Development Finance Corporation Limited,1.5525
5,GAIL (India) Limited,1.0425


## 4. Time influence  

- Finding the highly active time for the day, which may be useful to identify at what time of the day users are most active

In [12]:
q = '''match (tw:tweets)-[:at_this]->(t:time)
with t.year as year, t.month as month, t.day as day, t.hour as hour,
count(*) as num_tweets where toInteger(year) = 2022 and toInteger(month) = 6
and toInteger(day) = 2 and hour>"0"
return hour+' pm.', num_tweets'''

In [13]:
result = conn.query(q, db=db_config.DATABASE)

In [14]:
df = pd.DataFrame(result)
df.columns=['time','no_of_tweets']
display(HTML(df.head(10).to_html()))

,time,no_of_tweets
0,1 pm.,6
1,2 pm.,13
2,3 pm.,13
3,4 pm.,12
4,5 pm.,13
5,6 pm.,12
6,7 pm.,14
7,8 pm.,14
8,9 pm.,5
9,10 pm.,8


## 5. Most Influential Tweets
- Finding the companies associated with the most influenctial tweet, basically a tweet which has highest support count, companies about which most people are talking about

In [15]:
q = '''match (t:tweets)-[r:is_related_to]->(c:company)
with c.companyName as comp order by toInteger(t.support_Count) desc
return distinct(comp) as Popular_Companies limit 5'''

In [16]:
result = conn.query(q, db=db_config.DATABASE)

In [17]:
df = pd.DataFrame(result)
df.columns=['popular companies']
display(HTML(df.head().to_html()))

,popular companies
0,Reliance Industries Limited
1,ITC Limited
2,Gland Pharma Limited
3,Housing Development Finance Corporation Limited
4,Indian Oil Corporation Limited


# Conclusion
**Similarly many analytics can be done on the data using various algorithams to get the most out of our data and can help us tremendously to extend our business by costemizing pipelines for our cliens**